Before getting started, we need to import Python libraries, which consist of functionalities stored as functions.

For instance we will use BioPython. https://biopython.org/docs/1.75/api/Bio.html

You will see these import throughout the Notebooks of this course materials.<br/>
Make sure to always ensure these are being run.

In [ ]:
import os
import nglview
import shutil
from Bio.PDB import PDBList, MMCIFParser, Select, PDBIO
import os
from pathlib import Path

# local scripts
from scripts import viewer
from scripts.bio_align import *

We will now create a folder in which all the data will be stored.

In [ ]:
HOMEDIR = str(Path(_dh[0]))
os.chdir(HOMEDIR)
# We need to check whether the directory is there
try:
    os.mkdir('Bioinformatics')
except:
    print("Directory already exists")
os.chdir('Bioinformatics')

# Bioinformatics

### Analyzing your Drug target

It is important to get a good feeling for the target you are about to study. This is where literature research comes into play. A good starting point is the original publication that your PDB code refers to. This will also give you further references. In addition, general reviews (Pubmed, Google Scholar) can be very helpful.

1.	Describe the target (your report should contain at least the following information):
-	Pathology (why are we interested in the target?)
-	Clinical drugs available (which drugs are on the market)
 	Also use ATC information to further describe the drugs.
-	Protein information (what family of proteins is the target a member off, where is it located in the body?)

### Related proteins (off-targets), based on sequence

Using Bio-informatics we are going to compare the target to other proteins. We use the primary structure (sequence) of the protein for this. Go to: http://www.uniprot.org/

2.	You can find the link to Uniprot on the sequence tab on the page of your target on https://www.rcsb.org/

3.	Alternatively, find the name of the target in the ‘Protein Knowledgebase’ (UniprotKB). From the list of hits you should select the correct one.

4.	Verify the ‘Accession’ number with your supervisor. Yellow stars means that it is a correct and reviewed target, use a reviewed one unless you are working on H5N1.

In the case of the HIV targets, you have to make a sub selection within the POL polyprotein, do this by selecting the relevant domain in the list of domains below on the page.
RT -> select the ‘Reverse Transcriptase domain’, ‘Reverse Transcriptase Ribonuclease H’ domain.
Protease -> select the ‘Protease domain’

5.	When you have found the correct one, copy the following information:
    -	Sequence Length / Status / Protein Existence
    -	Mass (kDa)

6.	Use BLAST (Basic Local Alignment Search Tool, under ‘Advanced’) to find identical proteins in other species (related target 1).
-	Database ‘UniProtKB’, Threshold at 10, Matrix on auto, Filtering on none, Gapped on yes and Hits on 1000. Copy the following information.
    -	Length
    -	Identity
    -	Species

7.	Click on the ‘Accession’ number of the same hit, write the ‘Accession’ number down and copy the following information:
-	Status / Protein Existence
-	Mass (kDa)

8.	Use BLAST to find similar targets in the same species (related target 2).
-	Database ‘...Human’, Threshold at 1, Matrix on auto, Filtering on none, Gapped on yes and Hits on 1000. Copy the following information of the first hit that is not an isoform of your target.
    -	Length
    -	Identity
9.	Click on the ‘Accession’ number of the same hit, write the ‘Accession’ number down and copy the following information:
    -	Status / Protein Existence
    -	Mass (kDa)

Questions (1):
-	Which target is more similar compared to the original target?
-	Did you expect this?

Align the 3 proteins on Uniprot using the ‘Accession’ numbers.

In the overview, select the following tickboxes: “Similarity, Hydrophobic, Negative, Positive, Aromatic”, scroll down and make a screenshot (for report and presentation). Which targets are ‘more similar’ ?

### Related proteins (off-targets), based on structure

Since we are in the fortunate position that we have a crystal structure, we are also going to use a 3D similarity search. This is a structural similarity search rather than a sequence similarity search. Now we compare proteins based on their tertiary structure (3D structure)

10.	Start at http://www.rcsb.org/pdb/home/home.do  and find your target using the PDB identifier.
11.	Scroll down to ‘macromolecules’.
12.	Click ‘3D Structure’

<img src="img/LAB00_FIG00.png" alt="RCSB link" width="700"/>

13.	The top one should be your protein.
-	Write down the accession of the next most similar protein.
-	In the case of a GPCR target, ignore the T4Lysozyme hits.

# Retrieving a 3D structure

14.	Let's download the cystal structures of your protein of interest and visualize it.

    We have used 5IU4 as an example. If you execute the code block below, you will see a viewer with a 3D structure of the protein.

You can move around the structure with the mouse. Scrolling will zoom in/out, with the left mouse button you can rotate the structure. If you click the scrolling wheel you can move around the structure (sliding).

In [ ]:
TARGET_PDB_ID = "5IU4" # Enter your target PDB code here
PDB_CHAIN = 'A'
LIGAND_CODE = "ZMA" # Enter the ligand code here, example = 'zma'

target_pdb_path = download_structure(TARGET_PDB_ID, pdb_format='pdb', chain_id=PDB_CHAIN, ligand_to_keep=LIGAND_CODE)
view = nglview.show_structure_file(target_pdb_path, ext='pdb')
view.center()
view

I you see multiple molecules with balls and sticks, ask your TA to help remove one so that only one remains.

Let's save an image of the current 3D viewer, for your report/project presentation.

In [ ]:
view.render_image()

15. Now repeat for the most closely related structure. For the previous example, this was 8UGW.

In [ ]:
OFF_TARGET_PDB_ID = "8UGW" # Enter the PDB code of the structure most similar to your target protein

similar_pdb_path = download_structure(OFF_TARGET_PDB_ID, chain_id='A')
view = nglview.show_structure_file(similar_pdb_path)
view.center()
view

In [ ]:
view.render_image()

16. Let's align the two structures and visualize how well they do align.
- Note the RMSD below.

In [ ]:
structure1 = parse_structure(target_pdb_path)
structure2 = parse_structure(similar_pdb_path)
aligned_fpath, rmsd = AlignCystalStructures.by_coordinates(structure1, structure2)
print(f'The RMSD between the two structures is: {rmsd:.2f} Ångströms.')
view = nglview.show_structure_file(target_pdb_path)
view.add_component(aligned_fpath)
view.clear_representations(component=0)
view.clear_representations(component=1)
view.add_representation(
    repr_type='cartoon',
    selection='all',
    color='#a70f4a',
    component=0  # Target the first component
)
view.add_representation(
    repr_type='cartoon',
    selection='all',
    color='#269bcb',
    component=1  # Target the second component
)
view.center()
view


In [ ]:
view.render_image()

It is nice to have an overview of the structure, but since we are interested in designing new drugs, it makes more sense to have a closer look at the co-crystallized ligand. For this, we need to know the residue name of the ligand. This is a three-letter amino acid code that you can retrieve from the RCSB. In this case, the amino acid code is ZMA. First, we zoom in on the ligand.

In [ ]:
view = nglview.show_structure_file(target_pdb_path)
view.center(LIGAND_CODE)
view

In [ ]:
view.render_image()

Next, let's show the residues that are within 5 Ångstrom of the ligand

In [ ]:
view = nglview.show_structure_file(target_pdb_path)
view.center(LIGAND_CODE)
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

In [ ]:
view.render_image()

Have a look at the residues near the ligand, can you observe any important interactions? Describe in your report which interactions you observe, and what type of interactions they are.

17. Note that we do not see any hydrogen atoms, do you know why?

In the next stage, we will add the hydrogens and have another look at the structure. We will split the protein and ligand and save them separately.

In [ ]:
io = PDBIO()
io.set_structure(structure1)
io.save(f"ligand-{LIGAND_CODE}.pdb", ResSelect(LIGAND_CODE))
io.save(f"protein-{TARGET_PDB_ID}.pdb", NonHetSelect())

Now that we have separated ligands and protein, we can start adding hydrogens to the protein. We will use LePro for this, which is part of the LeDock program (https://en.wikipedia.org/wiki/LeDock).

21. Prepare the protein:

    Hydrogen atoms are added to the structure, because these are normally not resolved in the structure (due to limitations in resolution of the experimental method.

In [ ]:
command = f'"{_dh[0]}/bin/lepro" protein-{TARGET_PDB_ID}.pdb'
os.system(command)
shutil.move('pro.pdb', f'{TARGET_PDB_ID}_prepped.pdb')

In [ ]:
# combine protein and ligand files
filenames = [f'{TARGET_PDB_ID}_prepped.pdb',
             f"ligand-{LIGAND_CODE}.pdb"]

with open(f'{TARGET_PDB_ID}-complex.pdb', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

Let's have a look again at the protein

In [ ]:
with open(f'{TARGET_PDB_ID}-complex.pdb') as f:
    view = nglview.show_file(f, ext="pdb")

view.center(LIGAND_CODE)
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

In [ ]:
view.render_image()

23. Now, we will repeat the procedure for the most similar target that you identified (the highest scoring hit from the PDB):

In [ ]:
OFF_TARGET_LIGAND = "NGI"  # Enter the ligand code here, example = 'NGI'

io = PDBIO()
io.set_structure(structure2)
io.save(f"ligand-{OFF_TARGET_LIGAND}.pdb", ResSelect(OFF_TARGET_LIGAND))
io.save(f"protein-{OFF_TARGET_PDB_ID}.pdb", NonHetSelect())

In [ ]:
command = f'"{_dh[0]}/bin/lepro" protein-{OFF_TARGET_PDB_ID}.pdb'
os.system(command)
shutil.move('pro.pdb', f'{OFF_TARGET_PDB_ID}_prepped.pdb')

In [ ]:
# combine protein and ligand files
filenames = [f'{OFF_TARGET_PDB_ID}_prepped.pdb',
             f"ligand-{OFF_TARGET_LIGAND}.pdb"]

with open(f'{OFF_TARGET_PDB_ID}-complex.pdb', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

24. Prepare the protein

In [ ]:
with open(f'{OFF_TARGET_PDB_ID}-complex.pdb') as f:
    view = nglview.show_file(f, ext="pdb")

view.center(OFF_TARGET_LIGAND)
viewer.show_residues_around(view, selection=OFF_TARGET_LIGAND)
view

25. Now, let's try to align the structures. First we generate the alignment object

In [ ]:
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

import requests
data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{TARGET_PDB_ID}').json()[TARGET_PDB_ID.lower()]
SEQ1 = (data[0]['sequence'])
SEQ1 = Seq(SEQ1)

data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{OFF_TARGET_PDB_ID}').json()[OFF_TARGET_PDB_ID.lower()]
SEQ2 = (data[0]['sequence'])
SEQ2 = Seq(SEQ2)

alignments = pairwise2.align.globalxx(SEQ1, SEQ2)

for align1, align2, score, begin, end in alignments:
    filename = "alignment.fasta"
    with open(filename, "w") as handle:
        handle.write(">SEQ1\n%s\n>SEQ2\n%s\n" % (align1, align2))

print(alignments[0])

You can also align structure using https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html as you've seen previously.

You can also have a look at PLIP, which is a nice interaction profiler as well:
https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index

Finally, the interaction viewer on rcsb itself is also quite good, just type in your accession code in the search bar, scroll down to the ligand overview and click the ligand interaction box.

Toy around a bit with the structures and see if you can get a feeling for the binding site. In particular what kind of interactions seem to be important. Feel free to discuss this with your supervisor as well!

Another great way to look at the structure is on the rcsb website itself, where you can go to a 3D viewer of the ligand binding site by clicking on the Ligand interaction button.



This is the end of the bioinformatics part of the lab, for the report, please add a screenshot of the PLIP viewer. Try to identify key interactions in the binding site, either on the RCSB website or PLIP. Also think already how you might improve the ligand, can you think of additional interactions? The next two days, we will try to optimize the ligand. Tomorrow we will work with QSAR and cheminformatics. The day after we will use docking